In [ ]:
# 读取HY文件星下点数据
import numpy as np
import pandas as pd
import h5py
import glob
import os
from tqdm import tqdm
def read_HY1B_nadir(file):
    try:
        f = h5py.File(file, 'r')
        
        lat = np.array(f['Navigation Data/Latitude'][:,:])
        lon = np.array(f['Navigation Data/Longitude'][:,:])
        vza = np.array(f['Navigation Data/Satellite Zenith Angle'][:,:])
        vaa = np.array(f['Navigation Data/Satellite Azimuth Angle'][:,:])
        sza = np.array(f['Navigation Data/Solar Zenith Angle'][:,:])
        saa = np.array(f['Navigation Data/Solar Azimuth Angle'][:,:])
        
        
        out = os.path.basename(file)[17:25]
        # 宽范围搜索
        location = np.where((110<=lon)&(lon<=119)&(lat>=13)&(lat<=18)&(vza<=15))
        nir=(np.array(f['Geophysical Data/DN_865'][:,:]))[location]
        b = nir>600
        nir = nir*1.0
        nir[b] = np.nan
        mask = nir/nir
        
        lon = lon[location]*mask
        lat = lat[location]*mask
        vza = vza[location]*mask
        vaa = vaa[location]*mask
        sza = sza[location]*mask
        saa = saa[location]*mask
        
        lon = lon.flatten()
        lat = lat.flatten()
        vza = vza.flatten()
        vaa = vaa.flatten()
        sza = sza.flatten()
        saa = saa.flatten()
        
        data = {'lat':lat,
                'lon':lon,
                'vza':vza,
                'vaa':vaa,
                'sza':sza,
                'saa':saa
                }
        df = pd.DataFrame(data)
        gain = np.array(f['Calibration/Calibration Coefficients Scale factor'][:,:]).flatten()
        offset = np.array(f['Calibration/Calibration Coefficients Offsets factor'][:,:]).flatten()
        calibration_para={
        'calibration_coefficients_offset_factors': offset,
        'calibration_coefficients_scale_factors': gain,
        'time_dependent_correction_constant_terms': np.array(f['Calibration/Time-dependent Correction Constant Terms'][:,:]).flatten(),
        'time_dependent_correction_linear_coeficients': np.array(f['Calibration/Time-dependent Correction Linear Coefficients'][:,:]).flatten()
        }
        df_para = pd.DataFrame(calibration_para)
        
        bands = ['DN_412', 'DN_443', 'DN_490','DN_520','DN_565','DN_670','DN_750','DN_865']
        F0=[173.3835517,191.1745851,197.7310605,183.4959944,180.4321056,149.6390249,126.7797883,95.01985516]
        sza2 = np.array(f['Navigation Data/Solar Zenith Angle'][:,:])[location]*mask
        
        for i,band in enumerate(bands):
            radiance = (np.array(f['Geophysical Data/'+band][:,:]))[location]*mask*gain[i]+offset[i]
            nr = np.pi*radiance/F0[i]/np.cos(sza2*np.pi/180)
            nr = nr.flatten()
            band = 'b'+band[3:]
            df[band] = nr
            mean = np.nanmean(nr)  
            median = np.nanmedian(nr)  
            std = np.nanstd(nr)  
            # ptp = np.ptp(radiance[np.where(radiance < 500)])  # 最大最小值之差
            out=out+';'+str(mean)+','+str(median)+','+str(std)  #+','+str(ptp)
    except:
        out ='error'
        data = {'lat':[999],
                'lon':[999],
                'vza':[999],
                'vaa':[999],
                'sza':[999],
                'saa':[999]
                }
        df = pd.DataFrame(data)
        calibration_para={
        'calibration_coefficients_offset_factors': [999],
        'calibration_coefficients_scale_factors':[999],
        'time_dependent_correction_constant_terms':[999],
        'time_dependent_correction_linear_coeficients':[999]
         }
        df_para = pd.DataFrame(calibration_para)
    return out,df,df_para


if __name__ == '__main__':
    # 输入的书为HY1B L1A级
    filedir=r'G:\hyProject'
    files = glob.glob(filedir+os.sep+'H1B***H5')
    text = filedir+os.sep+'HY1B_nReflectanceTerm_.txt.txt'
    txt = open(text, 'w')  
    pbar = tqdm(total=len(files), desc='processing:')
    for i, file in enumerate(files):
        LT,df,df_para = read_HY1B_nadir(file)
        df.to_csv(path_or_buf=os.path.dirname(file)+os.sep+os.path.basename(file)[0:-3]+'_pixel_level_info.csv', index=False) # 写入csv文件
        df_para.to_csv(path_or_buf=os.path.dirname(file)+os.sep+os.path.basename(file)[0:-3]+'_calibration_parameters.csv', index=False) 
        txt.writelines(LT + '\n')
        if LT =='error':
            print('reading file: error')
        pbar.update(1)
    txt.close()
    pbar.close()

In [2]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
from bs4 import BeautifulSoup


class UntitledTestCase(unittest.TestCase):
    def setUp(self):
        # self.driver = webdriver.Firefox()
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(30)
        self.base_url = "https://www.baidu.com/"
        self.verificationErrors = []
        self.accept_next_alert = True

    def test_untitled_test_case(self):
        driver = self.driver
        driver.get(
            "https://ladsweb.modaps.eosdis.nasa.gov/search/order/4/MOD021KM--61,MOD03--61/2007-04-02..2007-04-02/D/117,19,119,15")
        # click select all
        time.sleep(5)
        ele1 = driver.find_element_by_id("tab4SelectAllLink")
        # print(ele.is_displayed())
        driver.execute_script("arguments[0].click();", ele1)
        # click review and order
        time.sleep(5)
        ele2 = driver.find_element_by_id("sub_btn_tab5")
        driver.execute_script("arguments[0].click();", ele2)
        # submit order
        time.sleep(5)
        ele3 = driver.find_element_by_id("tab5BtnSubmit")
        driver.execute_script("arguments[0].click();", ele3)
        # click find data
        driver.find_element_by_link_text("Find Data").click()
        # click
        time.sleep(10)
        ele4 = driver.find_element_by_id("btn_mainPast")
        driver.execute_script("arguments[0].click();", ele4)

        time.sleep(20)

        # 获取订单
        orderid = []
        dates = []
        rawurl = driver.current_url  # 获取当前URL
        driver.get(rawurl)  # 获取网页
        time.sleep(20)  # 等待加载完成
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')  # 解析网页
        # 抓取下单时间
        tags_dates = soup.find_all('div', class_='poOrderDate')
        for date in tags_dates:
            dates.append(date.contents[0])
        print(dates)

        # 抓取orderid
        tags_ids = soup.find_all('div', class_='poOrderItemStatus')
        for ids in tags_ids:
            orderid.append(ids.contents[0].split()[0])
        print(orderid)

    def is_element_present(self, how, what):
        try:
            self.driver.find_element(by=how, value=what)
        except NoSuchElementException as e:
            return False
        return True

    def is_alert_present(self):
        try:
            self.driver.switch_to_alert()
        except NoAlertPresentException as e:
            return False
        return True

    def close_alert_and_get_its_text(self):
        try:
            alert = self.driver.switch_to_alert()
            alert_text = alert.text
            if self.accept_next_alert:
                alert.accept()
            else:
                alert.dismiss()
            return alert_text
        finally:
            self.accept_next_alert = True

    def tearDown(self):
        self.driver.quit()
        self.assertEqual([], self.verificationErrors)


if __name__ == "__main__":
    
    
    unittest.main()


E
ERROR: C:\Users\lenovo\AppData\Roaming\jupyter\runtime\kernel-a2db9d33-57e2-4426-a762-e57f77f9fe1d (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\lenovo\AppData\Roaming\jupyter\runtime\kernel-a2db9d33-57e2-4426-a762-e57f77f9fe1d'

----------------------------------------------------------------------
Ran 1 test in 0.018s

FAILED (errors=1)


SystemExit: True

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
a = UntitledTestCase()